In [25]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
from bs4 import BeautifulSoup
import src.job_scrape_funcs as scrape
import src.analysis as ana
from datetime import datetime
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
def job_pipeline(job_title, location, post_time, pages, save_path, resume_path, word_scores):
    #TODO: try except statements
    # Scrape jobs:
    job_data = scrape.scrape_jobs(pages=pages, job_title=job_title, location=location, post_time=post_time)
    date = datetime.today().strftime("%Y-%m-%d")

    # Save:
    index = 1
    full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"
    while (Path.cwd()/full_path).exists():
        index += 1
        full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"

    print("Saving to ", full_path)
    with open(full_path, 'w', encoding='utf-8') as f:
        json.dump(job_data, f, ensure_ascii=False, indent=4)

    # Find matches:
    with open(full_path, 'r', encoding="utf-8") as f:
        data = json.load(f)

    raw_df = pd.DataFrame(data["jobs"])
    df = ana.find_job_match(raw_df, resume_path)
    print("Generating report:\n\n")
    ana.generate_html_report(df)
    return df

In [18]:
with open("data\job_queries\Data-Scientist_2025-04-08_3.json", 'r', encoding="utf-8") as f:
    job_data = pd.DataFrame(json.load(f)["jobs"])

<>:1: SyntaxWarning: invalid escape sequence '\j'
<>:1: SyntaxWarning: invalid escape sequence '\j'
C:\Users\tempu\AppData\Local\Temp\ipykernel_6384\76143991.py:1: SyntaxWarning: invalid escape sequence '\j'
  with open("data\job_queries\Data-Scientist_2025-04-08_3.json", 'r', encoding="utf-8") as f:


In [ ]:
# Scrape job listings:
jobs = scrape.scrape_jobs(pages=2, job_title="Data Scientist", location="Chicago", post_time=5)

In [8]:
# Save to JSON:
with open("data/job_queries/job_data_test.json", 'w', encoding='utf-8') as f:
    json.dump(jobs, f, ensure_ascii=False, indent=4)

In [19]:
# Load JSON:
with open("data/job_queries/Data-Scientist_2025-04-11_1.json", 'r', encoding='utf-8') as f:
    jobs = json.load(f)
raw_df = pd.DataFrame(jobs["jobs"])

In [21]:
word_scores = {
    "phd": 1.2,
    "python": 1.1,
    "senior": 0.9,
    "lead": 0.5,
}

In [26]:
# Find job matches:
df = ana.find_job_match(raw_df, "data/resume.pdf", word_scores=word_scores)

In [ ]:
df = job_pipeline(job_title="Data Scientist", 
                  location="Chicago", 
                  post_time=0.5, 
                  pages=1,
                  save_path="data/job_queries/", 
                  resume_path="data/resume.pdf",
                  word_scores=word_scores)

In [48]:
# generate job match report
html = ana.generate_html_report(df)

In [49]:
soup = BeautifulSoup(html)

In [50]:
with open("output1.html", "w") as file:
    file.write(str(soup))